# TODO:
MLP
- Data analysis
- Data preprocessing and normalization
- Data split
- Configuration
- Experiment tracking
- Experiments - meaningful based on the results of previous experiments
    - Including improvement techniques (e.g. Dropout, Normalization layers, Skip
    Connections, Bottleneck Layers, …)
- Hyperparameter search (grid/random) / sweep     #optional
- Results and evaluation metrics
- Clear code
- Markdown documentation and comments
- Final presentation of projects
- Effort on consultations


# OUR TODO:
- download dataset using pandas ? easier because of we can just use link but maybe everyone will use pandas 
- good enough data analysis that is seen in jupyter notebook, not just from reading dataset
- correlation chart  -  chart of correlation of columns between themselves
- outliers - what to do with them ? 
- counts of data in each column and plots of them 

imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Dataset

In [ ]:


df = pd.read_csv("dataset.csv")

print("Shape of dataset:", df.shape)
display(df.head())

numeric_df = df.select_dtypes(include=["number"])

# Data analysis with nice graphs and chart of correlations

In [ ]:
df.info()

df.count(numeric_only=True)

for col in numeric_df.columns:
    s = numeric_df[col]

    # Drop NaNs for clean counts
    counts = s.value_counts(dropna=False).sort_index()

    # X are just sequential positions (0,1,2,...)
    x = range(len(counts))
    y = counts.values
    labels = counts.index.astype(str)

    plt.figure(figsize=(8, 3))
    plt.bar(x, y, color="steelblue", alpha=0.9, width=0.8)
    plt.title(f"Count per value — {col}", fontsize=12, pad=8)
    plt.xlabel("Value")
    plt.ylabel("Count")
    plt.grid(True, linestyle="--", axis="y", alpha=0.4)

    # Show real values as x-tick labels (compressed)
    if len(labels) > 40:
        step = max(1, len(labels) // 40)
        plt.xticks(x[::step], labels[::step], rotation=45, ha="right")
    else:
        plt.xticks(x, labels, rotation=45, ha="right")

    plt.tight_layout()
    plt.show()

In [ ]:

corr = numeric_df.corr()

plt.figure(figsize=(10, 10))

sns.heatmap(
    corr,
    annot=True,         # Show correlation values
    cmap="coolwarm",    # Color palette
    fmt=".1f",          # Format numbers to 1
    square=True,        # Make cells square
    linewidths=1,       # Add thin lines between cells
    cbar_kws={"shrink": 0.7, "label": "Correlation"}
)

plt.title("Correlation Heatmap of Numeric Attributes", fontsize=14, pad=12)
plt.tight_layout()
plt.show()

# Data preprocessing and normalization

## Probably normalize like this: 
    - date_died change from date and 9999-99-99 with 1-died 0-alive ?? maybe
    - pregnant men don't exist, either leave 97 as NA for column pregnant for men or normalize and set it to 0